In [1]:
import langchain
from langchain_community.document_loaders import TextLoader,PyPDFLoader


e:\Projects\VSC\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
loader=PyPDFLoader("../data/docs/NVIDIA-2025-Annual-Report.pdf")
doc=loader.load()
full_text = "\n".join([doc.page_content for doc in doc])

with open("../data/docs/nvidia_10k.txt", "w", encoding="utf-8") as f:
    f.write(full_text)

In [4]:
print(doc[0].page_content[:1000])


2025
NVIDIA Corporation
Annual Review
Notice of Annual Meeting 
Proxy Statement 
Form 10-K


In [5]:
loader = TextLoader("../data/docs/nvidia_10k.txt", encoding="utf8")
data=loader.load()
data[0].metadata

{'source': '../data/docs/nvidia_10k.txt'}

In [6]:
from langchain_community.document_loaders import UnstructuredURLLoader
loader=UnstructuredURLLoader(urls=[
    "https://economictimes.indiatimes.com/markets/stocks/live-blog/bse-sensex-today-nifty50-stock-market-live-updates-gift-nifty-ril-wipro-share-price-16-january-2026/liveblog/126555529.cms?from=mdr",
    "https://investinglive.com/forex/indian-rupee-targeting-new-record-lows-as-rbis-interventions-continue-to-fail-20260116/"
])
data=loader.load()
data[0].page_content[:500]

"Benchmarks\n\nNifty25,694.3528.75\n\nFEATURED FUNDS\n\n★★★★★\n\nHSBC Large Cap Fund Direct-Growth\n\n5Y Return\n\n13.78 %\n\nInvest Now\n\nFEATURED FUNDS\n\n★★★★★\n\nMotilal Oswal Midcap Fund Direct-Growth\n\n5Y Return\n\n27.8 %\n\nInvest Now\n\nThe Economic Times\n\nET Markets\n\nEnglish EditionEnglish Editionहिन्दीગુજરાતીमराठीবাংলাಕನ್ನಡമലയാളംதமிழ்తెలుగు\n\n| Today's ePaper\n\nMy Watchlist Subscribe\n\nSign In\n\nBusiness News›Markets›Stocks›Live Blog›Market Highlights: Sensex settles 187 pts higher, Nifty above 25,650; TCS, HDFC Ban"

In [7]:
len(data[0].page_content)

36989

In [13]:
from langchain_text_splitters import  RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " "]
)
texts = text_splitter.split_documents(data)
len(texts)

59

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings

embedding_fn = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(doc, embedding_fn)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


In [15]:
test_docs = retriever.invoke("test query")
print(test_docs[0].page_content[:200])


Part IV
Item 15. Exhibits and Financial Statement Schedules
    Page
(a) 1. Financial Statements  
  Report of Independent Registered Public Accounting Firm (PCAOB ID: 238) 50
 
Consolidated Statement


In [19]:
from langchain_community.llms import Ollama

def build_prompt(context: str, query: str) -> str:
    return f"""
You are a financial analyst assistant.

Answer the question using ONLY the exact information present in the context.
Do NOT add explanations, examples, or background knowledge.
If the answer is not explicitly stated, say exactly:
Not found in documents.

Context:
{context}

Question:
{query}
"""


def rag_answer(query: str):
    # Retrieve relevant documents
    docs = retriever.invoke(query)

    # Combine retrieved context
    context = "\n\n".join(d.page_content for d in docs)

    # Build grounded prompt
    prompt = build_prompt(context, query)

    # Local LLM via Ollama (Mistral)
    llm = Ollama(
        model="ministral-3",
        temperature=0
    )

    return llm.invoke(prompt)


In [20]:
print(rag_answer("By what percentage did NVIDIA’s revenue grow year-over-year in Fiscal 2025?"))
print(rag_answer("What major industry shift drove NVIDIA’s performance in Fiscal 2025?"))
print(rag_answer("What is NVIDIA’s stock price today?"))

114%
The major industry shift that drove NVIDIA’s performance in Fiscal 2025 was the **sustained demand for exceptional 3D graphics and the scale of the gaming market**, along with the **growth in AI, data science, AV, robotics, and digital twin applications** enabled by accelerated computing and GPU architecture.

Specifically, **exceptional Data Center demand for the Hopper architecture** used for **large language models, recommendation engines, and generative AI applications** was a key driver.
Not found in documents.
